In [1]:
#第一次训练，将Unknown Label全部排除，构造WOE/IV

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns


import numpy as np
import pandas as pd
import tensorflow as tf

import atecml.data
import atecml.fs

from contextlib import contextmanager
from tqdm import tqdm
from time import strftime,time
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import joblib
from concurrent.futures import ProcessPoolExecutor

plt.style.use('ggplot')

In [3]:
#训练集为过滤掉所有unknown标签的
data= atecml.data.load_train()

predictors = [x for x in data.columns if x not in atecml.data.NOT_FEATURE_COLUMNS]
predictors.append('NaN_LIST')
target ='label'


#过滤unkown_record
train_df = data[data.label!=-1].reset_index(drop=True)

#F5可能是一个地域标签，聚类分组一下
train_df['f5'] =train_df['f5']//100

# 计算每个Feature的WOE/IV

In [4]:
iv_list = {}
woe_table = {}
type_list = {}

def mt_iv(feature):
    return atecml.data.calc_iv(df=train_df,feature=feature,target=target,category_num=64)

with ProcessPoolExecutor(max_workers=40) as pool:
    for pool_result in pool.map(mt_iv,predictors):
        iv = pool_result[0]
        woe_df = pool_result[1]
        feature_type = pool_result[2]
        iv_list.update(iv)
        woe_table.update(woe_df)
        type_list.update(feature_type)

[2018-07-03 13:44:29][WoE/IV Calculation for f4] Begin ...
[2018-07-03 13:44:29][WoE/IV Calculation for f5] Begin ...
[2018-07-03 13:44:29][WoE/IV Calculation for f2] Begin ...
[2018-07-03 13:44:29][WoE/IV Calculation for f1] Begin ...
[2018-07-03 13:44:29][WoE/IV Calculation for f3] Begin ...
[2018-07-03 13:44:29][WoE/IV Calculation for f6] Begin ...
[2018-07-03 13:44:29][WoE/IV Calculation for f7] Begin ...
[2018-07-03 13:44:29][WoE/IV Calculation for f9] Begin ...
[2018-07-03 13:44:29][WoE/IV Calculation for f12] Begin ...
[2018-07-03 13:44:29][WoE/IV Calculation for f8] Begin ...
[2018-07-03 13:44:29][WoE/IV Calculation for f11] Begin ...
[2018-07-03 13:44:29][WoE/IV Calculation for f10] Begin ...
[2018-07-03 13:44:29][WoE/IV Calculation for f17] Begin ...
[2018-07-03 13:44:29][WoE/IV Calculation for f18] Begin ...
[2018-07-03 13:44:29][WoE/IV Calculation for f21] Begin ...
[2018-07-03 13:44:29][WoE/IV Calculation for f19] Begin ...
[2018-07-03 13:44:29][WoE/IV Calculation for f20]

[2018-07-03 13:46:11][WoE/IV Calculation for f83] Begin ...
[2018-07-03 13:46:11][WoE/IV Calculation for f47] End   ...[Elapsed: 42.16s]
[2018-07-03 13:46:11][WoE/IV Calculation for f84] Begin ...
[2018-07-03 13:46:13][WoE/IV Calculation for f46] End   ...[Elapsed: 45.03s]
[2018-07-03 13:46:13][WoE/IV Calculation for f85] Begin ...
[2018-07-03 13:46:19][WoE/IV Calculation for f56] End   ...[Elapsed: 40.66s]
[2018-07-03 13:46:19][WoE/IV Calculation for f86] Begin ...
[2018-07-03 13:46:20][WoE/IV Calculation for f60] End   ...[Elapsed: 40.30s]
[2018-07-03 13:46:20][WoE/IV Calculation for f87] Begin ...
[2018-07-03 13:46:21][WoE/IV Calculation for f54] End   ...[Elapsed: 47.57s]
[2018-07-03 13:46:21][WoE/IV Calculation for f88] Begin ...
[2018-07-03 13:46:22][WoE/IV Calculation for f7] End   ...[Elapsed: 113.13s]
[2018-07-03 13:46:22][WoE/IV Calculation for f89] Begin ...
[2018-07-03 13:46:22][WoE/IV Calculation for f21] End   ...[Elapsed: 113.40s]
[2018-07-03 13:46:22][WoE/IV Calculation

[2018-07-03 13:47:41][WoE/IV Calculation for f90] End   ...[Elapsed: 78.87s]
[2018-07-03 13:47:41][WoE/IV Calculation for f143] Begin ...
[2018-07-03 13:47:44][WoE/IV Calculation for f111] End   ...[Elapsed: 44.45s]
[2018-07-03 13:47:44][WoE/IV Calculation for f144] Begin ...
[2018-07-03 13:47:44][WoE/IV Calculation for f89] End   ...[Elapsed: 82.16s]
[2018-07-03 13:47:44][WoE/IV Calculation for f145] Begin ...
[2018-07-03 13:47:45][WoE/IV Calculation for f115] End   ...[Elapsed: 41.67s]
[2018-07-03 13:47:45][WoE/IV Calculation for f146] Begin ...
[2018-07-03 13:47:46][WoE/IV Calculation for f109] End   ...[Elapsed: 50.65s]
[2018-07-03 13:47:46][WoE/IV Calculation for f147] Begin ...
[2018-07-03 13:47:47][WoE/IV Calculation for f114] End   ...[Elapsed: 43.75s]
[2018-07-03 13:47:47][WoE/IV Calculation for f148] Begin ...
[2018-07-03 13:47:47][WoE/IV Calculation for f101] End   ...[Elapsed: 69.98s]
[2018-07-03 13:47:47][WoE/IV Calculation for f149] Begin ...
[2018-07-03 13:47:49][WoE/IV 

[2018-07-03 13:49:17][WoE/IV Calculation for f112] End   ...[Elapsed: 137.76s]
[2018-07-03 13:49:17][WoE/IV Calculation for f202] Begin ...
[2018-07-03 13:49:19][WoE/IV Calculation for f172] End   ...[Elapsed: 43.49s]
[2018-07-03 13:49:19][WoE/IV Calculation for f203] Begin ...
[2018-07-03 13:49:20][WoE/IV Calculation for f173] End   ...[Elapsed: 43.11s]
[2018-07-03 13:49:20][WoE/IV Calculation for f204] Begin ...
[2018-07-03 13:49:21][WoE/IV Calculation for f122] End   ...[Elapsed: 118.73s]
[2018-07-03 13:49:21][WoE/IV Calculation for f205] Begin ...
[2018-07-03 13:49:22][WoE/IV Calculation for f169] End   ...[Elapsed: 47.88s]
[2018-07-03 13:49:22][WoE/IV Calculation for f206] Begin ...
[2018-07-03 13:49:23][WoE/IV Calculation for f167] End   ...[Elapsed: 49.32s]
[2018-07-03 13:49:23][WoE/IV Calculation for f207] Begin ...
[2018-07-03 13:49:25][WoE/IV Calculation for f160] End   ...[Elapsed: 67.37s]
[2018-07-03 13:49:25][WoE/IV Calculation for f208] Begin ...
[2018-07-03 13:49:26][WoE

[2018-07-03 13:50:35][WoE/IV Calculation for f227] End   ...[Elapsed: 41.17s]
[2018-07-03 13:50:35][WoE/IV Calculation for f261] Begin ...
[2018-07-03 13:50:36][WoE/IV Calculation for f215] End   ...[Elapsed: 57.88s]
[2018-07-03 13:50:36][WoE/IV Calculation for f262] Begin ...
[2018-07-03 13:50:37][WoE/IV Calculation for f208] End   ...[Elapsed: 72.18s]
[2018-07-03 13:50:37][WoE/IV Calculation for f263] Begin ...
[2018-07-03 13:50:38][WoE/IV Calculation for f234] End   ...[Elapsed: 35.40s]
[2018-07-03 13:50:38][WoE/IV Calculation for f264] Begin ...
[2018-07-03 13:50:39][WoE/IV Calculation for f222] End   ...[Elapsed: 52.13s]
[2018-07-03 13:50:39][WoE/IV Calculation for f265] Begin ...
[2018-07-03 13:50:39][WoE/IV Calculation for f233] End   ...[Elapsed: 38.04s]
[2018-07-03 13:50:39][WoE/IV Calculation for f266] Begin ...
[2018-07-03 13:50:40][WoE/IV Calculation for f221] End   ...[Elapsed: 53.73s]
[2018-07-03 13:50:40][WoE/IV Calculation for f267] Begin ...
[2018-07-03 13:50:42][WoE/I

[2018-07-03 13:52:11][WoE/IV Calculation for NaN_LIST] End   ...[Elapsed: 53.85s]
[2018-07-03 13:52:15][WoE/IV Calculation for f297] End   ...[Elapsed: 58.17s]


In [5]:
iv_df = pd.DataFrame()
iv_df['IV'] = pd.DataFrame.from_dict(iv_list,orient='index')[0]
iv_df['Type'] = pd.DataFrame.from_dict(type_list,orient='index')[0]
iv_df.sort_values('IV',ascending=False).head(100)

,IV,Type
f31,2.458742,category
f30,2.420329,category
f29,2.178416,category
f28,2.170980,category
f259,2.015321,value
f261,1.919631,value
f260,1.911540,value
f25,1.883468,category
f264,1.878833,value
f52,1.867689,category


In [6]:
iv_df.loc['NaN_LIST']

IV      0.758946
Type    category
Name: NaN_LIST, dtype: object

In [14]:
important_feature = list(iv_df[ (iv_df.IV > 0.3) & (iv_df.IV < 0.8)].index)

In [15]:
filter_feature = important_feature 
#list(set(important_feature) -set(atecml.data.NOT_IMP_FEATURE))
print(len(filter_feature))
rename_dict ={}
for item in filter_feature:
    old_feature_name = item
    new_feature_name = 'n_' + item
    rename_dict[old_feature_name] = new_feature_name

182


# 使用WOE/IV结果对原有训练集特征纬度提升

In [16]:
test_df = atecml.data.load_train()
test_df['f5'] =test_df['f5']//100

a = train_df['f5'].unique()
test_df['f5'][~test_df['f5'].isin(a)] =np.nan

def mt_conv_test(feature):
    return atecml.data.woe_convert(df=test_df,feature=feature,woe_table=woe_table,type_list=type_list,category_num=64)

new_feature_tt =[]
with ProcessPoolExecutor(max_workers=60) as pool:
    for pool_result in pool.map(mt_conv_test,predictors):
        new_feature_tt.append(pool_result)
        
_t_new_tt_df = pd.concat(new_feature_tt,axis=1)

_t_new_tt_df = _t_new_tt_df[filter_feature]

mean = _t_new_tt_df.mean(axis=1)
variance = _t_new_tt_df.var(axis=1)
skew = _t_new_tt_df.skew(axis=1)
_t_new_tt_df['mean'] = mean
#_t_new_tt_df['variance'] = variance
#_t_new_tt_df['skew'] = skew
_t_new_tt_df.rename(columns=rename_dict,inplace=True)
ext_tt_df = pd.concat([test_df,_t_new_tt_df],axis=1)
ext_tt_df.to_pickle('./01_train.dat')

In [17]:
ext_tt_df[ext_tt_df['label'] == -1]

,id,label,date,f1,f2,f3,f4,f5,f6,f7,...,n_f199,n_f244,n_f210,n_f177,n_f94,n_f160,n_f114,n_f181,n_f139,mean
274,3c4b3baa09510eec473f45079c7c7977fc01fdfed93a8b...,-1,2017-09-05,0,1,1,1,1007.0,0,3,...,0.435808,0.812497,-0.465065,0.404594,0.439580,0.560703,0.669359,0.397358,0.580074,0.456251
390,643b560780177054c375bd2ddd8a1ac6b99b728f016c75...,-1,2017-09-05,1,1,1,0,1008.0,0,3,...,0.434618,-0.349893,-0.042011,0.291907,-0.663949,-0.685810,-0.663949,0.578670,-0.663949,-0.296634
922,addd09c73d0bfa3d98e252865c3343b534db2f0701e089...,-1,2017-09-05,1,1,1,0,1008.0,0,4,...,0.434618,0.812497,-0.465065,0.404594,-1.753517,0.560703,0.360257,0.405429,0.433398,0.398064
1366,f8e66a38649c631bc205bd9f5cb953e214b17a3d9c80f2...,-1,2017-09-05,1,1,1,0,1008.0,0,6,...,-1.195379,0.812497,-1.195379,-1.195379,0.439580,-0.685810,0.165443,-1.195379,0.408186,-0.110014
1942,61dacd672d0d12112ae4c5a9f0aa7d348c9566ba60bda3...,-1,2017-09-05,1,0,0,1,1008.0,0,5,...,0.435808,0.812497,-0.246385,0.404594,0.780797,0.474129,0.360257,0.405429,0.395122,0.503511
1994,bbd743c945e39779d7136f90dcd6e005b04d6660dd65ec...,-1,2017-09-05,2,1,1,1,NaN,3,2,...,-1.195379,0.812497,-1.195379,-1.195379,-0.663949,-0.685810,-0.663949,-1.195379,-0.663949,-0.753707
2012,072a5f1007ec94ff4e6547783ec3dff4c72073dcfe5dfa...,-1,2017-09-05,0,1,0,0,1008.0,0,5,...,0.435808,0.812497,-0.246385,0.404594,0.780797,0.474129,0.434472,0.405429,0.558403,0.413907
2265,3842de26f717890858813543c6938c0fdb346b3e3fc715...,-1,2017-09-05,0,0,0,1,1008.0,1,4,...,0.434618,0.812497,-0.465065,0.404594,-1.753517,0.560703,0.360257,0.405429,0.433398,0.436581
2350,1ac41ee380516d13d2692b74eb08e7b33135fe8b745ca8...,-1,2017-09-05,0,0,0,1,1008.0,1,4,...,0.434618,-0.349893,-0.246385,0.404594,0.439580,0.560703,0.360257,0.397358,0.433398,0.322712
2470,1e06c933127058d1d88c63084522d4aa19dd0d55bc61ee...,-1,2017-09-05,1,0,0,1,1008.0,0,2,...,0.435808,-1.537289,0.291359,0.291907,-0.663949,-0.685810,-0.663949,0.578670,-0.663949,-0.331514


In [20]:
ext_tt_df[ext_tt_df['mean'] <0.5]['label'].sum()

7454

In [21]:
len(ext_tt_df[ext_tt_df['mean'] <0.5]['label'])

680387